# 1

In [ ]:
import pandas as pd
import gzip
import json
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
# from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import numpy as np

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import re

import pandas as pd
import gzip
import json
# Define the path to your .json.gz file
json_gz_file_path = '/kaggle/input/amazon-metadata/meta_Electronics.json'

# Define chunk size
chunksize = 10000  # Adjust this based on your system's capacity

# Open the compressed file and read it as a Pandas DataFrame in chunks
chunks = []
with open(json_gz_file_path, 'rb') as f:
    for chunk in pd.read_json(f, lines=True, chunksize=chunksize):
        chunks.append(chunk)

# Concatenate all chunks into a single DataFrame
meta_data = pd.concat(chunks, ignore_index=True)

# Define the path to your .json.gz file
json_gz_file_path = '/kaggle/input/electronics-dataset/Electronics_5.json'

# Define chunk size
chunksize = 10000  # Adjust this based on your system's capacity

# Open the compressed file and read it as a Pandas DataFrame in chunks
chunks = []
with open(json_gz_file_path, 'rb') as f:
    for chunk in pd.read_json(f, lines=True, chunksize=chunksize):
        chunks.append(chunk)

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunks, ignore_index=True)

# 2

In [ ]:


def cosine_similarity(X, Y):

    # Compute dot product
    dot_product = np.dot(X, Y.T)

    # Compute norms
    norm_X = np.linalg.norm(X, axis=1)
    norm_Y = np.linalg.norm(Y, axis=1)

    # Compute cosine similarity
    similarity = dot_product / (np.outer(norm_X, norm_Y) + 1e-10)  # Adding a small value to avoid division by zero

    return similarity

def find_similar_users_2(user_id, rating_matrix, val_vector, top_n):
    # Calculate cosine similarity between the given user (val_vector) and all users in the training set (rating_matrix)
    similarity = cosine_similarity(rating_matrix, val_vector.reshape(1, -1))
    
    # Sort the similarities and get the indices of top_n similar users
    similar_users = np.argsort(similarity[:, 0])[::-1][:top_n]
    
    return similar_users

def user_user_recommendation(rating_matrix, k_folds, top_ns):
    kf = KFold(n_splits=k_folds)
    mae_values = []
    
    for top_n in top_ns:
        fold_mae = []
        for train_index, val_index in kf.split(rating_matrix):
            train_set = rating_matrix.iloc[train_index]
            val_set = rating_matrix.iloc[val_index]
            # print(train_set.shape, val_set.shape)
            predicted_ratings = np.zeros_like(val_set)
            val_ratings = np.zeros_like(val_set)
            count = 0
            for user_id, user_ratings in val_set.iterrows():
                # print(count, "\\", len(val_set))
                similar_users = find_similar_users_2(user_id, train_set,user_ratings.values, top_n)
                avg_rating = train_set.iloc[similar_users].mean(axis=0)
                predicted_ratings[count] = avg_rating.values
                val_ratings[count] = user_ratings.values
                count += 1

                # print("--")
            break
                
        mae_values.append(mean_absolute_error(val_ratings[val_ratings != 0], predicted_ratings[val_ratings != 0]))
        print(top_n ,"-Done")
    return mae_values


def item_item_recommendation(rating_matrix, k_folds, top_ns):
    kf = KFold(n_splits=k_folds)
    mae_values = []
    
    for top_n in top_ns:
        fold_mae = []
        rating_matrix_transposed = rating_matrix.T
        # print(rating_matrix_transposed.info())
        for train_index, val_index in kf.split(rating_matrix_transposed):
            train_set = rating_matrix_transposed.iloc[train_index]
            val_set = rating_matrix_transposed.iloc[val_index]
            predicted_ratings = np.zeros_like(val_set)
            val_ratings = np.zeros_like(val_set)
            count = 0
            for user_id, user_ratings in val_set.iterrows():

                similar_users = find_similar_users_2(user_id, train_set,user_ratings.values, top_n)
                avg_rating = train_set.iloc[similar_users].mean(axis=0)
                predicted_ratings[count] = avg_rating.values
                val_ratings[count] = user_ratings.values
                count += 1
            break
                
        mae_values.append(mean_absolute_error(val_ratings[val_ratings != 0], predicted_ratings[val_ratings != 0]))
        print(top_n ,"-Done")
    return mae_values

# Function to convert lists and dictionaries to tuples
def convert_to_tuple(value):
    if isinstance(value, list):
        return tuple(value)
    elif isinstance(value, dict):
        return tuple(value.items())
    else:
        return value

In [ ]:
product_name = "Headphones"

# 3

In [ ]:

# Apply the conversion function to each element in the DataFrame
meta2 = meta_data.applymap(convert_to_tuple)

# Replace NaN values with a placeholder
meta2.fillna('NaN', inplace=True)

# Remove duplicates
meta2.drop_duplicates(inplace=True)

# Convert 'NaN' placeholder back to actual NaN values
meta2.replace('NaN', np.nan, inplace=True)

from PIL import Image, ImageEnhance

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Function to convert lists and dictionaries to tuples
def convert_to_tuple(value):
    if isinstance(value, list):
        return tuple(value)
    elif isinstance(value, dict):
        return tuple(value.items())
    else:
        return value

# Apply the conversion function to each element in the DataFrame
df = df.applymap(convert_to_tuple)

# Replace NaN values with a placeholder
df.fillna('NaN', inplace=True)

# Remove duplicates
df.drop_duplicates(inplace=True)

# Convert 'NaN' placeholder back to actual NaN values
df.replace('NaN', np.nan, inplace=True)

In [ ]:
product_meta_data_rows = []
i = 0
for index, row in meta2.iterrows():
    if product_name in row["title"]:
        product_meta_data_rows.append(i)
    i+=1


In [ ]:
m2 = meta2.iloc[product_meta_data_rows].dropna(subset = ["asin"])

In [ ]:
product_data =  pd.merge(m2, df, on='asin', how='inner')

# 4

In [ ]:
print("Number of Reviews:",len(product_data))

In [ ]:
avg_score = sum( product_data["overall"].values )/len(product_data)
print("Average Rating Score:",avg_score)

In [ ]:
print("Number of Unique Products:",len(product_data["asin"].unique()))

In [ ]:
print("Number of Good Ratings:", sum( [ 1 for index, row in product_data.iterrows() if row["overall"] >=3 ] ) )

In [ ]:
print("Number of Bad Ratings:", len(product_data) - sum( [ 1 for index, row in product_data.iterrows() if row["overall"] >=3 ] ) )

In [ ]:
print("Number of Reviews for each rating:")
i = 0
while i < 6:
    count = 0
    index = 0
    while index < len(product_data):
        row = product_data.iloc[index]
        if row["overall"] == i:
            count += 1
        index += 1
    print(f"Rating {i}: {count}")
    i += 1

# 5

In [ ]:
import pandas as pd
import re

def remove_accented_chars(text):
    if pd.isna(text):
        return text  # If NaN, return NaN
    return unidecode(text)


# Function to remove HTML tags
def remove_html_tags(text):
    if pd.isna(text):
        return text  # If NaN, return NaN
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# Function to remove special characters
def remove_special_characters(text):
    if pd.isna(text):
        return text
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

def lemmatize_text(text):
    if pd.isna(text):
        return text
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text


# Apply the function to the column
product_data['reviewText'] = product_data['reviewText'].apply(remove_html_tags)

In [ ]:
import pandas as pd
from unidecode import unidecode


# Apply the function to the column
product_data['reviewText'] = product_data['reviewText'].apply(remove_accented_chars)

In [ ]:
!pip install contractions

In [ ]:
import pandas as pd
import contractions

# Function to expand acronyms
def expand_acronyms(text):
    if pd.isna(text):
        return text
    return contractions.fix(text)

import pandas as pd
import spacy

# Load spaCy English model
nlp = spacy.load('en_core_web_sm')

# Apply the function to the column
product_data['reviewText'] = product_data['reviewText'].apply(expand_acronyms)

In [ ]:
import pandas as pd
import re



# Apply the function to the column
product_data['reviewText'] = product_data['reviewText'].apply(remove_special_characters)

In [ ]:


# Function to perform lemmatization

# Apply the function to the column
product_data['reviewText'] = product_data['reviewText'].apply(lemmatize_text)

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK stopwords (run this line only once)
import nltk
nltk.download('stopwords')

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(text):
    if pd.isna(text):
        return text
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

# Apply the function to the column
product_data['reviewText'] = product_data['reviewText'].apply(remove_stopwords)


# 6

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
from wordcloud import WordCloud
import matplotlib.pyplot as plt

print("Top 20 most reviewed brands:")
brand_reviews = { brand : 0 for brand in product_data["brand"].unique() }

for index, row in product_data.iterrows():
    brand_reviews[row["brand"]]+=1

top_20_brands = sorted( brand_reviews.items(), key = lambda x: x[1], reverse = True )[:20]

for brand, number_of_reviews in top_20_brands:
    print(brand,number_of_reviews)

In [ ]:
b2 = sorted( brand_reviews.items(), key = lambda x: x[1])[0:20]
print("Last 20 reviewed brands:")
for brand, number_of_reviews in b2:
    print(brand,number_of_reviews)


In [ ]:
import pandas as pd

average_ratings = product_data.groupby('title')['overall'].mean()

highest_rated_title = average_ratings.idxmax()

print("Headphones with the highest average rating:", highest_rated_title)

In [ ]:
good_reviews = product_data[product_data['overall'] >= 3]['reviewText']
bad_reviews = product_data[product_data['overall'] < 3]['reviewText']

# Tokenize the reviews into words
def tokenize_reviews(reviews):
    tokens = []
    for review in reviews:
        if pd.isna(review):
            tokens.extend([])
        else:
            tokens.extend(word_tokenize(review.lower()))
    return tokens

# Tokenize, remove stopwords and punctuation, and count the frequency of each word
good_tokens = tokenize_reviews(good_reviews)
bad_tokens = tokenize_reviews(bad_reviews)


# Generate frequency distributions
good_freq_dist = FreqDist(good_tokens)
bad_freq_dist = FreqDist(bad_tokens)

# Generate word clouds
good_wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(good_freq_dist)
bad_wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(bad_freq_dist)

# Plot word clouds
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.imshow(good_wordcloud, interpolation='bilinear')
plt.title('Word Cloud for Good Ratings')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(bad_wordcloud, interpolation='bilinear')
plt.title('Word Cloud for Bad Ratings')
plt.axis('off')

plt.show()

# Report most commonly used words for positive and negative reviews
print("Most commonly used words for positive reviews:")
print(good_freq_dist.most_common(10))

print("\nMost commonly used words for negative reviews:")
print(bad_freq_dist.most_common(10))

In [ ]:
ratings_number_of_reviews = dict()
i = 1
while i <= 5:
    count = 0
    index = 0
    while index < len(product_data):
        row = product_data.iloc[index]
        if row["overall"] == i:
            count += 1
        index += 1
    ratings_number_of_reviews[i] = count
    i += 1

data = ratings_number_of_reviews
# Get keys and values
labels = data.keys()
sizes = data.values()

# Plotting the pie chart
plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Ratings vs Number of Reviews')
plt.show()

In [ ]:
def extractYear(time):
    return int(time.split(",")[1].strip())
review_years = product_data["reviewTime"].apply(extractYear)

In [ ]:
product_data["reviewYear"] = review_years

In [ ]:
year_number_of_reviews = {year: 0 for year in review_years.unique()}
index = 0
while index < len(review_years):
    yn = review_years.iloc[index]
    year_number_of_reviews[yn] += 1
    index += 1

years = list(year_number_of_reviews.keys())
top_year = years[0]
max_reviews = year_number_of_reviews[top_year]
index = 1
while index < len(years):
    year = years[index]
    if year_number_of_reviews[year] > max_reviews:
        top_year = year
        max_reviews = year_number_of_reviews[year]
    index += 1

In [ ]:
year_number_of_customers = dict(product_data.groupby("reviewYear")["reviewerName"].unique())

In [ ]:
top_year = sorted(year_number_of_customers.items(), key = lambda x: len(x[1]),reverse = True)[0]

In [ ]:
top_year

# 7

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Tokenize the text in the 'reviewText' column
tokenized_reviews = product_data['reviewText'].dropna().apply(word_tokenize)

# Train the Word2Vec model
model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)

# Example usage: Get vector representation of a word
vector = model.wv['product']

# Example usage: Find most similar words
similar_words = model.wv.most_similar('product')

print("Vector representation of 'product':", vector)
print("Most similar words to 'product':", similar_words)

In [ ]:
def tokenize_review(text):
    if pd.isna(text):
        return []
    else:
        return word_tokenize(text)
def get_average_embedding(words, model):
    embeddings = [model.wv[word] for word in words if word in model.wv]
    if embeddings:
        return sum(embeddings) / len(embeddings)
    else:
        return [0] * model.vector_size  # Return zero vector if none of the words are in the vocabulary

# Get embeddings for each review
review_embeddings = [get_average_embedding(review, model) for review in product_data['reviewText'].apply(tokenize_review)]

In [ ]:
product_data["reviewTextEmbeddings"] = review_embeddings

# 8

In [ ]:
def review_class(overall):
    if overall>3:
        return "Good"
    elif overall == 3:
        return "Average"
    else:
        return "Bad"
product_data["ratingClass"] = [review_class(row["overall"]) for index,row in product_data.iterrows()]

In [ ]:
product_data['reviewText'] = product_data['reviewText'].str.replace(',', r'\,')

In [ ]:
import pickle as pkl
with open("Headphones.pkl","wb") as file:
    pkl.dump(product_data,file)

In [ ]:
import pickle as pkl

with open("/kaggle/input/headphones-dataset/Headphones.pkl","rb") as file:
    product_data_2 = pkl.load(file)

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

good_threshold = 4.0
bad_threshold = 3.0

product_df = product_data_2.dropna(subset=['overall', 'reviewText'])
product_df['Rating Class'] = product_df['overall'].apply(lambda x: 'Good' if x >= good_threshold else ('Average' if x == bad_threshold else 'Bad'))

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(product_df['Rating Class'])

label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:")
print(label_mapping)

tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(product_df['reviewText'])


indices_y0 = np.where(y == 0)[0]
indices_y1 = np.where(y == 1)[0]
indices_y2 = np.where(y == 2)[0]

num_indices_to_select = int(0.1 * len(indices_y2))

selected_indices_y2 = np.random.choice(indices_y2, size=num_indices_to_select, replace=False)

sampled_indices = np.concatenate([indices_y0, indices_y1, selected_indices_y2])

sampled_X = X[sampled_indices]
sampled_y = y[sampled_indices]
# print(sampled_y.shape, sampled_X.shape)

X_train, X_test, y_train, y_test = train_test_split(sampled_X, sampled_y, test_size=0.25, random_state=42)


In [ ]:


models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Support Vector Machine': KNeighborsClassifier(n_neighbors=4),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Gradient Boosting': GradientBoostingClassifier()
}

for name_model, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, target_names=['Bad', 'Average', 'Good'])
    print(f'Model: {name_model}')
    print(report)

In [ ]:
sampled_df = product_df.sample(frac=0.1, random_state=42) 
sampled_df.reset_index(drop=True, inplace=True)

sampled_df.info()
rating_matrix = sampled_df.pivot_table(index='reviewerID', columns='asin', values='overall')
min_rating = rating_matrix.min().min()
max_rating = rating_matrix.max().max()
normalized_rating_matrix = (rating_matrix - min_rating) / (max_rating - min_rating)
normalized_rating_matrix = normalized_rating_matrix.fillna(0)
non_zero_count = np.count_nonzero(normalized_rating_matrix)

print("Number of non-zero values in the DataFrame:", non_zero_count)

In [ ]:
top_ns = [10, 20, 30, 40, 50]
k_folds = 5
user_mae_values = user_user_recommendation(normalized_rating_matrix, k_folds ,top_ns)

In [ ]:
top_ns  = [10, 20, 30, 40, 50]
k_folds = 5
item_mae_values = item_item_recommendation(normalized_rating_matrix, k_folds ,top_ns)

In [ ]:
print(user_mae_values, item_mae_values)

In [ ]:
plt.plot([10, 20, 30, 40, 50], user_mae_values, label = "user-user")
plt.plot([10, 20, 30, 40, 50], item_mae_values, label = "item-item")
plt.legend()
plt.show()

In [ ]:
top_products = product_df.groupby('title')['overall'].sum().sort_values(ascending=False).head(10)
print("Top 10 products by user sum ratings:")
print(top_products)

In [ ]:
top_products = product_df.groupby('asin')['overall'].sum().sort_values(ascending=False).head(10)
print("Top 10 products by user sum ratings:")
print(top_products)

In [ ]:
top_products = df.groupby('asin')['overall'].sum().sort_values(ascending=False).head(10)

print("Top 10 products by user sum ratings:")
for asin in top_products.index:
    print(asin, meta_data_processed[meta_data_processed["asin"] == asin]["title"].values[0],top_products[asin])